# 支持向量机（SVM）

## 应用简化版SMO算法处理小规模数据集

In [1]:
import numpy as np


def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    with open(fileName) as fr:
        for line in fr.readlines():
            lineArr = line.strip().split('\t')
            dataMat.append([float(lineArr[0]), float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
    return dataMat, labelMat


def selectJrand(i, m):
    j = i
    while (j==i):
        j = int(np.random.uniform(0, m))
    return j


def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [2]:
dataArr, labelArr = loadDataSet('./testSet.txt')
print(labelArr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [3]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m, n = np.shape(dataMatrix)
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    while iter < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b  # Xi的预测值
            Ei = fXi - float(labelMat[i])  # Xi的预测误差
            if (labelMat[i] * Ei < -toler and alphas[i] < C) or (labelMat[i] * Ei > toler and alphas[i] > 0):
                j = selectJrand(i, m)
                fXj = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                if labelMat[i] != labelMat[j]:
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                
                if L == H:
                    print("L == H")
                    continue
                    
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - dataMatrix[i, :] * dataMatrix[i, :].T - \
                      dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0:
                    print('eta>=0')
                    continue
                
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if abs(alphas[j] - alphaJold) < 0.00001:
                    print("j not moving enough")
                    continue    
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[i, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j, :] * dataMatrix[j, :].T
                
                if 0 < alphas[i] and C > alphas[i]:
                    b = b1
                elif 0 < alphas[j] and C > alphas[j]:
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: {:d} i: {:d}, pairs changed {:d}".format(iter, i, alphaPairsChanged))
        if alphaPairsChanged == 0:  # 没有alpha对改变的时候就把iter加1，直到在maxIter次迭代内都没有alpha对改变时跳出循环
            iter += 1
        else:
            iter = 0  # 有alpha对改变的时候就把iter置0，直到在maxIter次迭代内都没有alpha对改变时跳出循环
        print("iteration: {:d}".format(iter))
    return b, alphas

In [4]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

L == H
iter: 0 i: 1, pairs changed 1
iter: 0 i: 3, pairs changed 2
L == H
j not moving enough
iter: 0 i: 6, pairs changed 3
iter: 0 i: 7, pairs changed 4
L == H
j not moving enough
L == H
j not moving enough
iter: 0 i: 17, pairs changed 5
j not moving enough
iter: 0 i: 22, pairs changed 6
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
iter: 0 i: 31, pairs changed 7
L == H
L == H
L == H
iter: 0 i: 51, pairs changed 8
j not moving enough
iter: 0 i: 54, pairs changed 9
L == H
j not moving enough
L == H
L == H
iter: 0 i: 69, pairs changed 10
j not moving enough
iteration: 0
j not moving enough
iter: 0 i: 7, pairs changed 1
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
L == H
iter: 0 i: 24, pairs changed 2
L == H
L == H
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 0
iter: 0 i: 1, 

iteration: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 3
j not moving enough
iter: 3 i: 23, pairs changed 1
j not moving enough
iter: 3 i: 52, pairs changed 2
j not moving enough
iteration: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 5
iter: 5 i: 17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iter: 5 i: 55, pairs changed 2
iteration: 0
j not moving enough
j not moving enough
j not moving enough
j not 

j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
iter: 1 i: 52, pairs changed 1
j not moving enough
j not moving enough
iteration: 0
j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
j not moving enough
j not moving enough
iteration: 2
j not moving enough
j not moving enough
j not moving enough
iteration: 3
j not moving enough
j not moving enough
j not moving enough
iteration: 4
j not moving enough
j not moving enough
j not moving enough
iteration: 5
j not moving enough
j not moving enough
j not moving enough
iteration: 6
j not moving enough
j not moving enough
j not moving enough
iteration: 7
j not moving enough
j not moving enough
j not moving enough
iteration: 8
j not moving enough
j not moving enough
j not moving enough
iteration: 9
j not moving enough
j not moving enough
j not moving enough
iteration: 10
j not moving enough
j not moving enough
j not moving enough
iteration: 11
j not moving enoug

iteration: 5
j not moving enough
j not moving enough
j not moving enough
iteration: 6
j not moving enough
j not moving enough
j not moving enough
iteration: 7
j not moving enough
j not moving enough
j not moving enough
iteration: 8
j not moving enough
j not moving enough
j not moving enough
iteration: 9
j not moving enough
j not moving enough
j not moving enough
iteration: 10
j not moving enough
j not moving enough
j not moving enough
iteration: 11
j not moving enough
j not moving enough
j not moving enough
iteration: 12
j not moving enough
j not moving enough
j not moving enough
iteration: 13
j not moving enough
j not moving enough
j not moving enough
iteration: 14
j not moving enough
j not moving enough
j not moving enough
iteration: 15
j not moving enough
j not moving enough
j not moving enough
iteration: 16
j not moving enough
j not moving enough
j not moving enough
iteration: 17
j not moving enough
j not moving enough
j not moving enough
iteration: 18
j not moving enough
j not mov

j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration: 5
j not moving enough
j not moving enough
iter: 5 i: 52, pairs changed 1
iteration: 0
j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
j not moving enough
j not moving enough
iteration: 2
j not moving enough
j not moving enough
j not moving enough
iteration: 3
j not moving enough
j not moving enough
iter: 3 i: 29, pairs changed 1
j not moving enough
iteration: 0
j not moving enough
j not moving enough
j not moving enough
iteration: 1
j not moving enough
j not moving enough
j not moving en

j not moving enough
iteration: 40


In [5]:
b

matrix([[-3.76166215]])

In [6]:
alphas[alphas > 0]

matrix([[0.13536712, 0.21274032, 0.01618813, 0.36429556]])

In [7]:
np.shape(alphas[alphas > 0])[1]

4

In [8]:
for i in range(100):
    if alphas[i] > 0:
        print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0


In [9]:
w = np.multiply(alphas, np.mat(labelArr).T).T * np.mat(dataArr)
w

matrix([[ 0.80225189, -0.27809246]])

## 利用完整Platt SMO算法加速优化

In [10]:
class optStruct:
    
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))
        
        
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b  # Xk的预测值
    Ek = fXk - float(oS.labelMat[k])
    return Ek


def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej


def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

In [11]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i] * Ei < -oS.tol and oS.alphas[i] < oS.C) or \
       (oS.labelMat[i] * Ei > oS.tol and oS.alphas[i] > 0):
        j, Ej = selectJ(i, oS, Ei)  # 如果是第一次循环的话，那么就随机选择一个alpha值
        
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L == H")
            return 0
        
        eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T
        if eta >= 0:
            print("eta >= 0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            print("j not moving enough")
            return 0
        
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i, :].T - \
             oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j, :].T - \
             oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[j]:
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [12]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)
                print("fullSet, iter: {:d} i: {:d}, pairs changed {:d}".format(iter, i, alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < oS.C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("non-bound, iter: {:d} i: {:d}, pairs changed {:d}".format(iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:  # 遍历整个数据集都未对任意alpha对进行修改时就退出循环
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print("iteration number: {:d}".format(iter))
    return oS.b, oS.alphas

In [13]:
dataArr, labelArr = loadDataSet('./testSet.txt')
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

fullSet, iter: 0 i: 0, pairs changed 1
fullSet, iter: 0 i: 1, pairs changed 1
fullSet, iter: 0 i: 2, pairs changed 2
fullSet, iter: 0 i: 3, pairs changed 2
fullSet, iter: 0 i: 4, pairs changed 2
fullSet, iter: 0 i: 5, pairs changed 3
fullSet, iter: 0 i: 6, pairs changed 3
fullSet, iter: 0 i: 7, pairs changed 3
j not moving enough
fullSet, iter: 0 i: 8, pairs changed 3
fullSet, iter: 0 i: 9, pairs changed 3
j not moving enough
fullSet, iter: 0 i: 10, pairs changed 3
fullSet, iter: 0 i: 11, pairs changed 3
fullSet, iter: 0 i: 12, pairs changed 3
fullSet, iter: 0 i: 13, pairs changed 3
fullSet, iter: 0 i: 14, pairs changed 3
fullSet, iter: 0 i: 15, pairs changed 3
fullSet, iter: 0 i: 16, pairs changed 3
fullSet, iter: 0 i: 17, pairs changed 4
fullSet, iter: 0 i: 18, pairs changed 5
fullSet, iter: 0 i: 19, pairs changed 5
fullSet, iter: 0 i: 20, pairs changed 5
fullSet, iter: 0 i: 21, pairs changed 5
j not moving enough
fullSet, iter: 0 i: 22, pairs changed 5
fullSet, iter: 0 i: 23, pairs 

In [14]:
b

matrix([[-3.75442552]])

In [15]:
alphas[alphas > 0]

matrix([[0.00064605, 0.05410545, 0.02783825, 0.06053798, 0.18654537,
         0.02516125, 0.05748445, 0.19135091]])

In [16]:
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i], X[i, :].T)
    return w

In [17]:
ws = calcWs(alphas, dataArr, labelArr)
ws

array([[ 0.7929368 ],
       [-0.16014993]])

In [18]:
datMat = np.mat(dataArr)
datMat[0] * np.mat(ws) + b

matrix([[-1.26213896]])

In [19]:
labelArr[0]

-1.0

In [20]:
datMat[2] * np.mat(ws) + b

matrix([[2.48648633]])

In [21]:
labelArr[2]

1.0

In [22]:
datMat[1] * np.mat(ws) + b

matrix([[-1.77004164]])

In [23]:
labelArr[1]

-1.0

## 在复杂数据集上应用核函数

### 在训练中使用核函数

In [24]:
def kernelTrans(X, A, kTup):
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = np.exp(K / -1 * kTup[1] ** 2)
    else:
        raise NameError('Houston We have a Problem -- That Kernel is not recognized')
    return K

In [25]:
class optStruct:
    
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))
        # 矩阵K记录的是记过高维映射后，高维映射空间中两个向量之间的内积，K[i, j]表示的就是第i个和第j个高维空间中两个向量的内积
        # 由Mercer定理可知，核函数K(a, b)连续且对称即：K(a,b) = K(b, a)
        # 只不过使用的是核技巧：在原始空间中，使用核函数来对两个原始向量做数值计算
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)

In [26]:
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * oS.K[:, k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

In [27]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i] * Ei < -oS.tol and oS.alphas[i] < oS.C) or \
       (oS.labelMat[i] * Ei > oS.tol and oS.alphas[i] > 0):
        j, Ej = selectJ(i, oS, Ei)  # 如果是第一次循环的话，那么就随机选择一个alpha值
        
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L == H")
            return 0
        
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]
        if eta >= 0:
            print("eta >= 0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            print("j not moving enough")
            return 0
        
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] - \
             oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j] - \
             oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[j]:
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [28]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)
                print("fullSet, iter: {:d} i: {:d}, pairs changed {:d}".format(iter, i, alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < oS.C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("non-bound, iter: {:d} i: {:d}, pairs changed {:d}".format(iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:  # 遍历整个数据集都未对任意alpha对进行修改时就退出循环
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print("iteration number: {:d}".format(iter))
    return oS.b, oS.alphas

### 在测试中使用核函数

In [29]:
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('./testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 1000, ('rbf', k1))
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("there are {:d} Support Vectors".format(np.shape(sVs)[0]))
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        # 计算支持向量与要被分类的特征向量之间在高维特征空间中的内积，因为要计算的是训练误差，所以可以等价于oS.K[svInd, i]
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        # predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        predict = np.multiply(alphas[svInd], labelSV).T * kernelEval + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: {:f}".format(float(errorCount) / m))
    dataArr, labelArr = loadDataSet('./testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        # 要计算的是测试误差
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))  # 高维空间特征空间中特征向量之间的内积
        predict = np.multiply(alphas[svInd], labelSV).T * kernelEval + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: {:f}".format(float(errorCount) / m))

In [30]:
testRbf()

L == H
fullSet, iter: 0 i: 0, pairs changed 0
fullSet, iter: 0 i: 1, pairs changed 1
fullSet, iter: 0 i: 2, pairs changed 2
fullSet, iter: 0 i: 3, pairs changed 3
fullSet, iter: 0 i: 4, pairs changed 4
fullSet, iter: 0 i: 5, pairs changed 5
fullSet, iter: 0 i: 6, pairs changed 6
fullSet, iter: 0 i: 7, pairs changed 6
fullSet, iter: 0 i: 8, pairs changed 7
fullSet, iter: 0 i: 9, pairs changed 7
fullSet, iter: 0 i: 10, pairs changed 8
fullSet, iter: 0 i: 11, pairs changed 9
fullSet, iter: 0 i: 12, pairs changed 9
fullSet, iter: 0 i: 13, pairs changed 10
fullSet, iter: 0 i: 14, pairs changed 11
L == H
fullSet, iter: 0 i: 15, pairs changed 11
fullSet, iter: 0 i: 16, pairs changed 11
fullSet, iter: 0 i: 17, pairs changed 11
fullSet, iter: 0 i: 18, pairs changed 11
fullSet, iter: 0 i: 19, pairs changed 11
fullSet, iter: 0 i: 20, pairs changed 11
L == H
fullSet, iter: 0 i: 21, pairs changed 11
fullSet, iter: 0 i: 22, pairs changed 11
fullSet, iter: 0 i: 23, pairs changed 11
L == H
fullSet, it

In [31]:
testRbf(0.1)  # k1 = 0.1该参数此时减少了每个支持向量的影响程度，因此需要更多的支持向量

fullSet, iter: 0 i: 0, pairs changed 1
L == H
fullSet, iter: 0 i: 1, pairs changed 1
L == H
fullSet, iter: 0 i: 2, pairs changed 1
fullSet, iter: 0 i: 3, pairs changed 2
fullSet, iter: 0 i: 4, pairs changed 2
L == H
fullSet, iter: 0 i: 5, pairs changed 2
L == H
fullSet, iter: 0 i: 6, pairs changed 2
fullSet, iter: 0 i: 7, pairs changed 2
fullSet, iter: 0 i: 8, pairs changed 2
fullSet, iter: 0 i: 9, pairs changed 2
fullSet, iter: 0 i: 10, pairs changed 2
L == H
fullSet, iter: 0 i: 11, pairs changed 2
fullSet, iter: 0 i: 12, pairs changed 2
fullSet, iter: 0 i: 13, pairs changed 3
L == H
fullSet, iter: 0 i: 14, pairs changed 3
fullSet, iter: 0 i: 15, pairs changed 4
L == H
fullSet, iter: 0 i: 16, pairs changed 4
fullSet, iter: 0 i: 17, pairs changed 5
fullSet, iter: 0 i: 18, pairs changed 6
fullSet, iter: 0 i: 19, pairs changed 6
j not moving enough
fullSet, iter: 0 i: 20, pairs changed 6
fullSet, iter: 0 i: 21, pairs changed 7
L == H
fullSet, iter: 0 i: 22, pairs changed 7
L == H
fullSet

j not moving enough
fullSet, iter: 3 i: 95, pairs changed 0
j not moving enough
fullSet, iter: 3 i: 96, pairs changed 0
L == H
fullSet, iter: 3 i: 97, pairs changed 0
j not moving enough
fullSet, iter: 3 i: 98, pairs changed 0
L == H
fullSet, iter: 3 i: 99, pairs changed 0
iteration number: 4
there are 34 Support Vectors
the training error rate is: 0.440000
the test error rate is: 0.510000


## 手写识别问题回顾

In [32]:
from os import listdir


def img2vector(filename):
    returnVect = np.zeros((1, 1024))
    with open(filename) as fr:
        for i in range(32):
            lineStr = fr.readline()
            for j in range(32):
                returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect


def loadImages(dirName):
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i, :] = img2vector("{!s}/{!s}".format(dirName, fileNameStr))
    return trainingMat, hwLabels

In [33]:
def testDigits(kTup=('rbf', 10)):
    dataArr, labelArr = loadImages('./trainingDigits/')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A>0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("there are {:d} Support Vectors".format(np.shape(sVs)[0]))
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = np.multiply(alphas[svInd], labelSV).T * kernelEval + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: {:f}".format(float(errorCount) / m))
    dataArr, labelArr = loadImages('./testDigits/')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = np.multiply(alphas[svInd], labelSV).T * kernelEval + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: {:f}".format(float(errorCount) / m))

In [34]:
# testDigits(('rbf', 10)) 需要训练很久

### Tips and Tests

In [35]:
a = np.mat([[1, 0, 3], [3, 4, 5]])
a

matrix([[1, 0, 3],
        [3, 4, 5]])

In [36]:
a.A > 0

array([[ True, False,  True],
       [ True,  True,  True]])

In [37]:
a.A < 4

array([[ True,  True,  True],
       [ True, False, False]])

In [38]:
(a.A > 0) * (a.A < 4)

array([[ True, False,  True],
       [ True, False, False]])

In [39]:
np.nonzero((a.A > 0) * (a.A < 4))

(array([0, 0, 1]), array([0, 2, 0]))

In [40]:
np.nonzero((a.A > 0) * (a.A < 4))[0]

array([0, 0, 1])